# 要約 
このJupyter Notebookは、Gemma 2 9bモデルを使用して埋め込みを計算し、それに基づいて分類器をトレーニングする準備を行うものです。ノートブックは、主に埋め込みの取得に焦点を当てています。

## 問題設定
ユーザーは、Chatbot Arenaのデータを用いて、異なるモデルが生成した応答に基づいてユーザーの好みを予測するための埋め込みを計算することを目的としています。具体的には、Gemma 2モデルを利用して、プロンプトと2つのモデルの応答を組み合わせたテキストから埋め込みを生成します。これにより、機械学習モデルのトレーニングデータとして使えるベクトル表現を作成します。

## 使用する手法やライブラリ
このノートブックでは、以下のライブラリと手法が用いられています。

1. **ライブラリのインポート**:
   - `bitsandbytes`: メモリ効率を向上させるためのライブラリ。
   - `transformers`: Hugging Faceのライブラリを使用し、Gemma 2モデルおよびトークナイザーを利用。
   - `torch`: PyTorchライブラリを用いた深層学習。
   - `pandas`、`numpy`、`matplotlib`: データ操作や可視化のためのライブラリ。

2. **モデルの設定**:
   - Gemma 2モデルを4ビット量子化し、2つのGPUデバイスを使用してモデルの異なるインスタンスをロードします。

3. **データ前処理**:
   - 提供されたトレーニングデータを読み込み、プロンプトと応答を整形します。

4. **トークン化**:
   - 整形したテキストをトークナイズし、PyTorchのテンソル形式に変換します。

5. **埋め込みの計算**:
   - モデルを用いてトークン化されたデータから埋め込みを計算。
   - 並行処理を利用し、2つのスレッドで異なるデバイスを活用して埋め込みを取得します。

6. **結果の保存**:
   - 計算された埋め込みを保存し、さらにトレーニングデータをCSV形式で保存します。

このノートブックは、Chatbot Arenaコンペティションのデータを活用し、LLM（大規模言語モデル）を使用したユーザーの好み予測へ向けた埋め込み計算を効率的に行うための基盤を提供しています。

---


# 用語概説 
以下に、Jupyter Notebookの内容に関して、機械学習や深層学習の初心者がつまずきやすい専門用語の簡単な解説を示します。この解説は、あまり一般的でない用語や、このノートブック特有のドメイン知識に焦点を当てています。

1. **埋め込み (Embeddings)**:
   - 文や単語等のデータを、数値ベクトル（通常は高次元）の形式で表現する方法。深層学習モデルにおいては、この数値ベクトルを用いて意味的な情報を保持しつつ、計算を効率化します。

2. **量子化 (Quantization)**:
   - モデルのサイズを小さくし、計算速度を向上させるために、浮動小数点数を低ビットの整数表現に変換する技術。ここでは「4ビット」と「float16」という形式が用いられています。

3. **アテンションマスク (Attention Mask)**:
   - トランスフォーマー型モデルにおいて、入力シーケンスのどの部分に注意を向けるべきかを指定するマスク。特に、パディングされた部分を無視するために使用されます。

4. **自動混合精度計算 (Automatic Mixed Precision - AMP)**:
   - 計算の精度を高めつつ、モデルのトレーニングや推論を高速化するために、異なるデータ型（例えば、`float16` と `float32`）を自動的に使い分ける技術。

5. **ガーベジコレクション (Garbage Collection)**:
   - プログラムが使用しているメモリを自動的に管理し、不要になったメモリを開放する仕組み。大規模データ処理ではメモリ管理が重要です。

6. **スレッド (Thread)**:
   - プログラム内での異なる作業を同時に実行するための単位。並行処理により、処理時間を短縮することを目的としています。

7. **トークナイズ (Tokenization)**:
   - テキストを単語や部分に分割するプロセス。このプロセスによって、テキストデータがモデルに入力できる形式に変換されます。

8. **メモリ効率的SDP (Memory-Efficient SDP)**:
   - 深層学習モデルのメモリ使用効率を高めるための特定の設定や技術。ここでは具体的な設定が変更されています。

9. **隠れ状態 (Hidden States)**:
   - トランスフォーマーモデル内での中間的な出力。各層で生成され、モデルの内部状態を保存しています。

10. **平均プーリング (Mean Pooling)**:
    - インプットのデータに対する平均を計算して特徴ベクトルを得る操作。特に埋め込みの取得時に利用されます。

これらの定義を通じて、初心者でもNotebookの内容をよりよく理解できるようになると思います。

---


<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Gemma 2 - 9b 

We use Gemma 2 9b model to get embeddings and train a classifier on it. This is first part and in this we only compute embed. You can also use other models. Let's get started!

Upvote if you found this helpful!

</div>
<div class="column-right">

# 日本語訳

## Gemma 2 - 9b 

Gemma 2 9bモデルを使用して埋め込みを取得し、それに基づいて分類器をトレーニングします。これはその第一部であり、ここでは埋め込みのみを計算します。他のモデルを使用することもできます。それでは始めましょう！

役に立ったら、ぜひアップボートしてください！

# ライブラリのインポート

</div>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Import libs 

</div>
<div class="column-right">

# 日本語訳

# bitsandbytesライブラリをインストールします
!pip install -q -U bitsandbytes 

# Hugging FaceのtransformersライブラリをGitHubからインストールします
!pip install -q git+https://github.com/huggingface/transformers

# SentencePieceライブラリをインストールします（テキスト処理に使用します）
!pip install sentencepiece

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install -q -U bitsandbytes 
!pip install -q git+https://github.com/huggingface/transformers
!pip install sentencepiece
```

</div>
<div class="column-right">

# 日本語訳

```python
# 必要なライブラリをインポートします
import os  # OS関連の機能を提供します
import gc  # ガーベジコレクションを扱うためのライブラリです
import re  # 正規表現操作用ライブラリです
from time import time  # 時間を計測するための機能をインポートします

import torch  # PyTorchライブラリをインポートします
import transformers  # transformersライブラリをインポートします
import sklearn  # scikit-learnライブラリをインポートします（機械学習用）
import random  # ランダム数生成用ライブラリをインポートします
import numpy as np  # 数値計算用ライブラリをインポートします
import pandas as pd  # データ操作用ライブラリをインポートします
import matplotlib.pyplot as plt  # プロット作成用ライブラリをインポートします

from transformers import Gemma2ForCausalLM, GemmaTokenizer, BitsAndBytesConfig  # Gemmaモデルとトークナイザーをインポート

import time  # 時間計測用ライブラリを再度インポート
from torch.cuda.amp import autocast  # 自動混合精度計算用のautocast機能をインポート
from threading import Thread  # スレッド処理用ライブラリをインポート

# CUDAのメモリ効率的なSDPを有効化します。この設定により、GPUのメモリ使用効率が改善されます
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# CUDAが利用可能でない場合、GPUが必要であることを知らせるメッセージを表示します
# if (not torch.cuda.is_available()): print("Sorry - GPU required!")
```

</div>
</details>

In [ ]:
# 必要なライブラリをインポートします
import os  # OS関連の機能を提供します
import gc  # ガーベジコレクションを扱うためのライブラリです
import re  # 正規表現操作用ライブラリです
from time import time  # 時間を計測するための機能をインポートします

import torch  # PyTorchライブラリをインポートします
import transformers  # transformersライブラリをインポートします
import sklearn  # scikit-learnライブラリをインポートします（機械学習用）
import random  # ランダム数生成用ライブラリをインポートします
import numpy as np  # 数値計算用ライブラリをインポートします
import pandas as pd  # データ操作用ライブラリをインポートします
import matplotlib.pyplot as plt  # プロット作成用ライブラリをインポートします

from transformers import Gemma2ForCausalLM, GemmaTokenizer, BitsAndBytesConfig  # Gemmaモデルとトークナイザーをインポート

import time  # 時間計測用ライブラリを再度インポート
from torch.cuda.amp import autocast  # 自動混合精度計算用のautocast機能をインポート
from threading import Thread  # スレッド処理用ライブラリをインポート

# CUDAのメモリ効率的なSDPを有効化します。この設定により、GPUのメモリ使用効率が改善されます
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# CUDAが利用可能でない場合、GPUが必要であることを知らせるメッセージを表示します
# if (not torch.cuda.is_available()): print("Sorry - GPU required!")

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import os
import gc
import re
from time import time

import torch
import transformers
import sklearn
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from transformers import Gemma2ForCausalLM, GemmaTokenizer, BitsAndBytesConfig

import time
from torch.cuda.amp import autocast
from threading import Thread

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# if (not torch.cuda.is_available()): print("Sorry - GPU required!")
```

</div>
<div class="column-right">

# 日本語訳

```python
# 設定
```

</div>
</details>

# 設定

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Configs 

</div>
<div class="column-right">

# 日本語訳

# 設定クラスを定義します
class CFG:
    # モデルのパスを指定します（Kaggle上のGemma 2 9bモデルのパス）
    MODEL_PATH = '/kaggle/input/gemma-2-9b-hf'
    # 最大入力シーケンスの長さを設定します
    MAX_LENGTH = 1024
    # バッチサイズを設定します
    BATCH_SIZE = 2
    
# 使用するデバイスを指定します（GPUの0番目のデバイス）
device0 = torch.device('cuda:0')
# 使用するデバイスを指定します（GPUの1番目のデバイス）
device1 = torch.device('cuda:1')

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
class CFG:
    MODEL_PATH = '/kaggle/input/gemma-2-9b-hf'
    MAX_LENGTH = 1024
    BATCH_SIZE = 2
    
device0 = torch.device('cuda:0')
device1 = torch.device('cuda:1')
```

</div>
<div class="column-right">

# 日本語訳

```python
# モデルの読み込み
```

</div>
</details>

# モデルの読み込み

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Load model

</div>
<div class="column-right">

# 日本語訳

# トークナイザーを指定したモデルパスから読み込みます
tokenizer = GemmaTokenizer.from_pretrained(CFG.MODEL_PATH)

# 4ビットの量子化設定を定義します
bnb_config_4bit = BitsAndBytesConfig(
    load_in_4bit=True,  # 4ビットでモデルを読み込みます
    bnb_4bit_compute_dtype=torch.float16,  # 計算データ型をfloat16に指定します
    bnb_4bit_use_double_quant=False)  # 二重量子化を使用するかどうかを指定します（ここでは無効にします）

# GPUの0番目のデバイスにモデルを読み込みます
model_0 = Gemma2ForCausalLM.from_pretrained(CFG.MODEL_PATH,
                                        revision="float16",  # モデルのバージョンをfloat16に指定
                                        device_map='cuda:0',  # モデルをデバイス0にマッピング
                                        quantization_config=bnb_config_4bit)  # 量子化設定を適用します

# GPUの1番目のデバイスにモデルを読み込みます
model_1 = Gemma2ForCausalLM.from_pretrained(CFG.MODEL_PATH,
                                        revision="float16",  # モデルのバージョンをfloat16に指定
                                        device_map='cuda:1',  # モデルをデバイス1にマッピング
                                        quantization_config=bnb_config_4bit)  # 量子化設定を適用します

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
tokenizer = GemmaTokenizer.from_pretrained(CFG.MODEL_PATH)

bnb_config_4bit = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False)

model_0 = Gemma2ForCausalLM.from_pretrained(CFG.MODEL_PATH,
                                        revision="float16",
                                        device_map='cuda:0',
                                        quantization_config=bnb_config_4bit)        

model_1 = Gemma2ForCausalLM.from_pretrained(CFG.MODEL_PATH,
                                        revision="float16",
                                        device_map='cuda:1',
                                        quantization_config=bnb_config_4bit)     
```

</div>
<div class="column-right">

# 日本語訳

```python
# トレーニングデータの準備
```

</div>
</details>

# トレーニングデータの準備

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Prepare train 

</div>
<div class="column-right">

# 日本語訳

# 入力文字列を処理する関数を定義します
def process(input_str):
    # 角括弧を剥がし、文字列を分割します
    stripped_str = input_str.strip('[]')
    # 文章を抽出し、各文の前後のダブルクォートを削除します
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    # 最後の文を返します。もし文がなければ空文字を返します
    return sentences[-1] if sentences else ''
  
# トレーニングデータをCSVファイルから読み込みます
train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena-additional-data-90k-columns/Merged_data.csv')

# 各列のプロンプトと応答を処理して整形します
train.loc[:, 'prompt'] = train['prompt'].apply(process)  # プロンプトを処理
train.loc[:, 'response_a'] = train['response_a'].apply(process)  # モデルAの応答を処理
train.loc[:, 'response_b'] = train['response_b'].apply(process)  # モデルBの応答を処理

# モデルのインプットテキストを整形します
train['text'] = '<start_of_turn>User prompt: ' + train['prompt'] +  '\n\nModel A :\n' + train['response_a'] +'\n\n----\n\nModel B:\n'  + train['response_b'] + '<end_of_turn><eos>'  # モデルのプロンプトと応答の形式を設定

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return sentences[-1] if sentences else ''
  
train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena-additional-data-90k-columns/Merged_data.csv')

train.loc[:, 'prompt'] = train['prompt'].apply(process)
train.loc[:, 'response_a'] = train['response_a'].apply(process)
train.loc[:, 'response_b'] = train['response_b'].apply(process)


train['text'] = '<start_of_turn>User prompt: ' + train['prompt'] +  '\n\nModel A :\n' + train['response_a'] +'\n\n----\n\nModel B:\n'  + train['response_b'] + '<end_of_turn><eos>'
```

</div>
<div class="column-right">

# 日本語訳

```python
# データセットからサンプルを40,000件のみ取得します
train = train[:40000]

# トレーニングデータの最初の1行を表示します
train.head(1)  # データフレームの最初の1行を表示して中身を確認します
```

</div>
</details>

In [ ]:
# データセットからサンプルを40,000件のみ取得します
train = train[:40000]

# トレーニングデータの最初の1行を表示します
train.head(1)  # データフレームの最初の1行を表示して中身を確認します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# taking only 40k sample
train = train[:40000]
train.head(1)
```

</div>
<div class="column-right">

# 日本語訳

```python
# インデックス10のトレーニングデータテキストを表示します
print(train['text'][10])  # 特定のテキストの内容を確認するために出力します
```

</div>
</details>

In [ ]:
# インデックス10のトレーニングデータテキストを表示します
print(train['text'][10])  # 特定のテキストの内容を確認するために出力します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
print(train['text'][10])
```

</div>
<div class="column-right">

# 日本語訳

```python
# トークナイズ（トークン化）
```

</div>
</details>

# トークナイズ（トークン化）

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Tokenize 

</div>
<div class="column-right">

# 日本語訳

# トレーニングテキストをトークナイズします
tokens = tokenizer(train['text'].tolist(),
                   padding='max_length',  # 最大長さにパディングを施します
                   max_length=CFG.MAX_LENGTH,  # 最大シーケンス長を設定します
                   truncation=True,  # 長すぎるテキストは切り捨てます
                   return_tensors='pt')  # PyTorchテンソルとして戻します

# トークン化された入力IDとアテンションマスクを取得します
INPUT_IDS = tokens['input_ids']  # 入力ID
ATTENTION_MASKS = tokens['attention_mask']  # アテンションマスク

# 新しいDataFrameを作成します
data = pd.DataFrame()
data['INPUT_IDS'] = [tensor.tolist() for tensor in INPUT_IDS]  # 入力IDをリストに変換
data['ATTENTION_MASKS'] = [tensor.tolist() for tensor in ATTENTION_MASKS]  # アテンションマスクをリストに変換

# 最初の2行を表示して内容を確認します
data[:2]

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
tokens = tokenizer(train['text'].tolist(),
                   padding='max_length',
                   max_length=CFG.MAX_LENGTH,
                   truncation=True,
                   return_tensors='pt')

INPUT_IDS = tokens['input_ids']
ATTENTION_MASKS = tokens['attention_mask']

data = pd.DataFrame()
data['INPUT_IDS'] = [tensor.tolist() for tensor in INPUT_IDS]
data['ATTENTION_MASKS'] = [tensor.tolist() for tensor in ATTENTION_MASKS]
data[:2]
```

</div>
<div class="column-right">

# 日本語訳

```python
# 埋め込みの計算
```

</div>
</details>

# 埋め込みの計算

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Compute embedding

</div>
<div class="column-right">

# 日本語訳

# 埋め込みを計算する関数を定義します
def get_embeddings(df, model, device, batch_size=CFG.BATCH_SIZE):  
    # INPUT_IDSとATTENTION_MASKSをTensorに変換します
    input_ids = torch.tensor(df['INPUT_IDS'].values.tolist(), dtype=torch.long)
    attention_mask = torch.tensor(df['ATTENTION_MASKS'].values.tolist(), dtype=torch.long)

    embed_list = []  # 埋め込みを格納するリストを初期化

    # バッチサイズに基づいてデータフレームを処理します
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終了インデックスを計算
        batch_input_ids = input_ids[start_idx:end_idx].to(device)  # バッチの入力IDをデバイスに転送
        batch_attention_mask = attention_mask[start_idx:end_idx].to(device)  # バッチのアテンションマスクをデバイスに転送
        gc.collect()  # ガーベジコレクションを実行してメモリを解放
        torch.cuda.empty_cache()  # GPUメモリのキャッシュをクリア

        with torch.no_grad():  # 勾配を計算しないコンテキストで実行
            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask, output_hidden_states=True)  # モデルに入力
            embed = outputs.hidden_states[-1]  # 最後の隠れ状態を取得
            embed_mean = torch.mean(embed, dim=1).cpu()  # 平均プーリングを行いCPUに戻す
            embed_list.append(embed_mean)  # 埋め込みをリストに追加
            
            torch.cuda.empty_cache()  # GPUメモリのキャッシュをクリア
        
    # リスト内のすべての埋め込みを結合します
    embeddings = torch.cat(embed_list, dim=0)
    return embeddings  # 計算された埋め込みを返します

# 埋め込みを計算するための関数を定義します
def compute_embed(df, model, device, results, index):
    results[index] = get_embeddings(df, model, device)  # 結果を指定されたインデックスに保存します

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def get_embeddings(df, model, device, batch_size=CFG.BATCH_SIZE):  
    input_ids = torch.tensor(df['INPUT_IDS'].values.tolist(), dtype=torch.long)
    attention_mask = torch.tensor(df['ATTENTION_MASKS'].values.tolist(), dtype=torch.long)

    embed_list = []

    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        batch_input_ids = input_ids[start_idx:end_idx].to(device)
        batch_attention_mask = attention_mask[start_idx:end_idx].to(device)
        gc.collect()
        torch.cuda.empty_cache()
        with torch.no_grad():
            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask, output_hidden_states=True)
            embed = outputs.hidden_states[-1]
            embed_mean = torch.mean(embed, dim=1).cpu() #mean pool
            embed_list.append(embed_mean) 
            
            torch.cuda.empty_cache()
        
    embeddings = torch.cat(embed_list, dim=0)
    return embeddings

def compute_embed(df, model, device, results, index):
    results[index] = get_embeddings(df, model, device)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 処理開始のタイムスタンプを記録します
st = time.time()

# データのサンプル数を取得します
N_SAMPLES = len(data)
# サンプル数の半分を計算します
half = round(N_SAMPLES / 2)
# データフレームを2つの部分に分割します
sub1 = data.iloc[0:half].copy()  # 前半のサブセット
sub2 = data.iloc[half:N_SAMPLES].copy()  # 後半のサブセット

results = {}  # 埋め込みを格納するための辞書を初期化

# スレッドを使用して並行処理を行います
t0 = Thread(target=compute_embed, args=(sub1, model_0, device0, results, 0))  # モデル0を使用するスレッド
t1 = Thread(target=compute_embed, args=(sub2, model_1, device1, results, 1))  # モデル1を使用するスレッド

# スレッドを開始します
t0.start()
t1.start()

# スレッドの終了を待ちます
t0.join()
t1.join()

# 処理が完了したことを知らせるメッセージをプリントします
print(f"Processing complete. Total time: {time.time() - st:.2f} seconds")  # 処理にかかった合計時間を表示
```

</div>
</details>

In [ ]:
# 処理開始のタイムスタンプを記録します
st = time.time()

# データのサンプル数を取得します
N_SAMPLES = len(data)
# サンプル数の半分を計算します
half = round(N_SAMPLES / 2)
# データフレームを2つの部分に分割します
sub1 = data.iloc[0:half].copy()  # 前半のサブセット
sub2 = data.iloc[half:N_SAMPLES].copy()  # 後半のサブセット

results = {}  # 埋め込みを格納するための辞書を初期化

# スレッドを使用して並行処理を行います
t0 = Thread(target=compute_embed, args=(sub1, model_0, device0, results, 0))  # モデル0を使用するスレッド
t1 = Thread(target=compute_embed, args=(sub2, model_1, device1, results, 1))  # モデル1を使用するスレッド

# スレッドを開始します
t0.start()
t1.start()

# スレッドの終了を待ちます
t0.join()
t1.join()

# 処理が完了したことを知らせるメッセージをプリントします
print(f"Processing complete. Total time: {time.time() - st:.2f} seconds")  # 処理にかかった合計時間を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
st = time.time()

N_SAMPLES = len(data)
half = round(N_SAMPLES / 2)
sub1 = data.iloc[0:half].copy()
sub2 = data.iloc[half:N_SAMPLES].copy()

results = {}

t0 = Thread(target=compute_embed, args=(sub1, model_0, device0, results, 0))
t1 = Thread(target=compute_embed, args=(sub2, model_1, device1, results, 1))

t0.start()
t1.start()

t0.join()
t1.join()

print(f"Processing complete. Total time: {time.time() - st:.2f} seconds")
```

</div>
<div class="column-right">

# 日本語訳

```python
# 2つのスレッドから得られた埋め込みを結合します
embeddings = torch.cat([results[0], results[1]], dim=0)

# 結合された埋め込みの形状を表示します
embeddings.shape  # 埋め込みのテンソルの形状を出力して確認します
```

</div>
</details>

In [ ]:
# 2つのスレッドから得られた埋め込みを結合します
embeddings = torch.cat([results[0], results[1]], dim=0)

# 結合された埋め込みの形状を表示します
embeddings.shape  # 埋め込みのテンソルの形状を出力して確認します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
embeddings = torch.cat([results[0], results[1]], dim=0)
embeddings.shape
```

</div>
<div class="column-right">

# 日本語訳

```python
# ガーベジコレクションを実行してメモリを解放します
gc.collect()

# 使用が終了したモデルを削除してメモリを解放します
del model_1  # モデル1を削除
del model_0  # モデル0を削除

# GPUメモリのキャッシュをクリアします
torch.cuda.empty_cache()  # メモリの効率的な使用のためにキャッシュをクリアします
```

</div>
</details>

In [ ]:
# ガーベジコレクションを実行してメモリを解放します
gc.collect()

# 使用が終了したモデルを削除してメモリを解放します
del model_1  # モデル1を削除
del model_0  # モデル0を削除

# GPUメモリのキャッシュをクリアします
torch.cuda.empty_cache()  # メモリの効率的な使用のためにキャッシュをクリアします

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
gc.collect()
del model_1
del  model_0
torch.cuda.empty_cache()
```

</div>
<div class="column-right">

# 日本語訳

```python
# 埋め込みの保存
```

</div>
</details>

# 埋め込みの保存

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Save embed

</div>
<div class="column-right">

# 日本語訳

# 埋め込みを保存するパスを指定します
save_path = 'gemma2_train_embed.npy'

# 埋め込みを.npyファイルとして保存します
np.save(save_path, embeddings.numpy())  # NumPy形式で埋め込みを保存

# 完全性のためにトレーニングデータも保存します
train.to_csv('train_embed.csv', index=False)  # トレーニングデータをCSVファイルとして保存

# 保存完了メッセージを表示します
print(f"Concatenated embeddings saved to {save_path}")  # 保存されたファイルのパスを表示します

</div>